# **Extracting Information from Legal Documents Using RAG**

## **Objective**

The main objective of this assignment is to process and analyse a collection text files containing legal agreements (e.g., NDAs) to prepare them for implementing a **Retrieval-Augmented Generation (RAG)** system. This involves:

* Understand the Cleaned Data : Gain a comprehensive understanding of the structure, content, and context of the cleaned dataset.
* Perform Exploratory Analysis : Conduct bivariate and multivariate analyses to uncover relationships and trends within the cleaned data.
* Create Visualisations : Develop meaningful visualisations to support the analysis and make findings interpretable.
* Derive Insights and Conclusions : Extract valuable insights from the cleaned data and provide clear, actionable conclusions.
* Document the Process : Provide a detailed description of the data, its attributes, and the steps taken during the analysis for reproducibility and clarity.

The ultimate goal is to transform the raw text data into a clean, structured, and analysable format that can be effectively used to build and train a RAG system for tasks like information retrieval, question-answering, and knowledge extraction related to legal agreements.

### **Business Value**  


The project aims to leverage RAG to enhance legal document processing for businesses, law firms, and regulatory bodies. The key business objectives include:

* Faster Legal Research: <br> Reduce the time lawyers and compliance officers spend searching for relevant case laws, precedents, statutes, or contract clauses.
* Improved Contract Analysis: <br> Automatically extract key terms, obligations, and risks from lengthy contracts.
* Regulatory Compliance Monitoring: <br> Help businesses stay updated with legal and regulatory changes by retrieving relevant legal updates.
* Enhanced Decision-Making: <br> Provide accurate and context-aware legal insights to assist in risk assessment and legal strategy.


**Use Cases**
* Legal Chatbots
* Contract Review Automation
* Tracking Regulatory Changes and Compliance Monitoring
* Case Law Analysis of past judgments
* Due Diligence & Risk Assessment

## **1. Data Loading, Preparation and Analysis** <font color=red> [20 marks] </font><br>

### **1.1 Data Understanding**

The dataset contains legal documents and contracts collected from various sources. The documents are present as text files (`.txt`) in the *corpus* folder.

There are four types of documents in the *courpus* folder, divided into four subfolders.
- `contractnli`: contains various non-disclosure and confidentiality agreements
- `cuad`: contains contracts with annotated legal clauses
- `maud`: contains various merger/acquisition contracts and agreements
- `privacy_qa`: a question-answering dataset containing privacy policies

The dataset also contains evaluation files in JSON format in the *benchmark* folder. The files contain the questions and their answers, along with sources. For each of the above four folders, there is a `json` file: `contractnli.json`, `cuad.json`, `maud.json` `privacy_qa.json`. The file structure is as follows:

```
{
    "tests": [
        {
            "query": <question1>,
            "snippets": [{
                    "file_path": <source_file1>,
                    "span": [ begin_position, end_position ],
                    "answer": <relevant answer to the question 1>
                },
                {
                    "file_path": <source_file2>,
                    "span": [ begin_position, end_position ],
                    "answer": <relevant answer to the question 2>
                }, ....
            ]
        },
        {
            "query": <question2>,
            "snippets": [{<answer context for que 2>}]
        },
        ... <more queries>
    ]
}
```

### **1.2 Load and Preprocess the data** <font color=red> [5 marks] </font><br>

#### Loading libraries

In [24]:
## The following libraries might be useful
!pip install -q langchain-openai
!pip install -U -q langchain-community
!pip install -U -q langchain-chroma
!pip install -U -q datasets
!pip install -U -q ragas
!pip install -U -q rouge_score

In [25]:
# Import essential libraries
import os
import glob
import re
import string
import random
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns
import json
import logging
from tqdm import tqdm
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hrida\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### **1.2.1** <font color=red> [3 marks] </font>
Load all `.txt` files from the folders.

You can utilise document loaders from the options provided by the LangChain community.

Optionally, you can also read the files manually, while ensuring proper handling of encoding issues (e.g., utf-8, latin1). In such case, also store the file content along with metadata (e.g., file name, directory path) for traceability.

In [26]:
# Load the files as documents

def load_documents(corpus_dir):
    documents = []
    metadata = []
    for root, dirs, files in os.walk(corpus_dir):
        for file in files:
            if file.endswith('.txt'):
                file_path = os.path.join(root, file)
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        text = f.read()
                except UnicodeDecodeError:
                    with open(file_path, 'r', encoding='latin1') as f:
                        text = f.read()
                documents.append(text)
                metadata.append({'file_name': file, 'dir_path': root, 'file_path': file_path})
    return documents, metadata

# Set the path to your corpus directory
corpus_dir = 'rag_legal/corpus'  # Update this path as needed

documents, metadata = load_documents(corpus_dir)
print(f"Loaded {len(documents)} documents.")

Loaded 698 documents.


In [27]:
# To list the contents of the corpus directory
print(os.listdir("rag_legal"))

['benchmarks', 'corpus']


#### **1.2.2** <font color=red> [2 marks] </font>
Preprocess the text data to remove noise and prepare it for analysis.

Remove special characters, extra whitespace, and irrelevant content such as email and telephone contact info.
Normalise text (e.g., convert to lowercase, remove stop words).
Handle missing or corrupted data by logging errors and skipping problematic files.

In [28]:
# Clean and preprocess the data

def clean_text(text):
    # Remove emails
    text = re.sub(r'\S+@\S+', '', text)
    # Remove phone numbers
    text = re.sub(r'\b\d{10,}\b', '', text)
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Lowercase
    text = text.lower()
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in text.split() if word not in stop_words]
    return ' '.join(words)

cleaned_documents = []
error_files = []
for i, doc in enumerate(documents):
    try:
        cleaned = clean_text(doc)
        cleaned_documents.append(cleaned)
    except Exception as e:
        logging.error(f"Error cleaning document {metadata[i]['file_path']}: {e}")
        error_files.append(metadata[i]['file_path'])

print(f"Cleaned {len(cleaned_documents)} documents. Skipped {len(error_files)} files.")

Cleaned 698 documents. Skipped 0 files.


### **1.3 Exploratory Data Analysis** <font color=red> [10 marks] </font><br>

#### **1.3.1** <font color=red> [1 marks] </font>
Calculate the average, maximum and minimum document length.

In [29]:
# Calculate the average, maximum and minimum document length.

if cleaned_documents:
    doc_lengths = [len(doc.split()) for doc in cleaned_documents]
    avg_length = np.mean(doc_lengths)
    max_length = np.max(doc_lengths)
    min_length = np.min(doc_lengths)

    print(f"Average document length: {avg_length:.2f} words")
    print(f"Maximum document length: {max_length} words")
    print(f"Minimum document length: {min_length} words")
else:
    print("No cleaned documents available to calculate lengths.")

Average document length: 8621.62 words
Maximum document length: 82546 words
Minimum document length: 132 words


#### **1.3.2** <font color=red> [4 marks] </font>
Analyse the frequency of occurrence of words and find the most and least occurring words.

Find the 20 most common and least common words in the text. Ignore stop words such as articles and prepositions.

In [30]:
# Find frequency of occurence of words

all_words = ' '.join(cleaned_documents).split()
word_counts = Counter(all_words)

most_common = word_counts.most_common(20)
least_common = word_counts.most_common()[:-21:-1]

print("20 Most common words:")
for word, count in most_common:
    print(f"{word}: {count}")

print("\n20 Least common words:")
for word, count in least_common:
    print(f"{word}: {count}")

20 Most common words:
company: 148175
shall: 107998
agreement: 104577
section: 75368
parent: 58010
party: 49664
date: 39300
time: 35256
material: 34211
merger: 33848
subsidiaries: 33318
b: 31719
applicable: 31370
including: 29399
respect: 28848
may: 28114
stock: 26654
information: 25687
parties: 24614
business: 23531

20 Least common words:
newer: 1
peoplefuncom: 1
nonmarketing: 1
httpwwwyouronlinechoiceseu: 1
httpwwwaboutadsinfochoices: 1
checkins: 1
httpsvunglecomprivacypolicy: 1
vungle: 1
httpswwwtapresearchcomuserprivacy: 1
tapresearch: 1
httpsdevtapjoycomfaqtapjoyprivacypolicy: 1
tapjoy: 1
httpswwwstartappcompolicyprivacypolicy: 1
startappcom: 1
httpaboutsoomlaenduserprivacypolicy: 1
soomla: 1
httpswwwsmaatocomprivacy: 1
smaato: 1
httpspinsightmediacomprivacy: 1
pinsight: 1


#### **1.3.3** <font color=red> [4 marks] </font>
Analyse the similarity of different documents to each other based on TF-IDF vectors.

Transform some documents to TF-IDF vectors and calculate their similarity matrix using a suitable distance function. If contracts contain duplicate or highly similar clauses, similarity calculation can help detect them.

Identify for the first 10 documents and then for 10 random documents. What do you observe?

In [31]:
# Transform the page contents of documents

# Use TF-IDF to vectorize the documents
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(cleaned_documents)

# Compute similarity matrix for the first 10 documents
similarity_matrix_first10 = cosine_similarity(tfidf_matrix[:10])

print("Similarity matrix for first 10 documents:")
print(np.round(similarity_matrix_first10, 2))

Similarity matrix for first 10 documents:
[[1.   0.16 0.63 0.42 0.5  0.6  0.61 0.7  0.44 0.47]
 [0.16 1.   0.16 0.1  0.12 0.14 0.13 0.15 0.14 0.12]
 [0.63 0.16 1.   0.52 0.64 0.71 0.72 0.76 0.4  0.57]
 [0.42 0.1  0.52 1.   0.43 0.49 0.5  0.49 0.23 0.38]
 [0.5  0.12 0.64 0.43 1.   0.56 0.57 0.58 0.29 0.47]
 [0.6  0.14 0.71 0.49 0.56 1.   0.72 0.63 0.35 0.56]
 [0.61 0.13 0.72 0.5  0.57 0.72 1.   0.66 0.36 0.55]
 [0.7  0.15 0.76 0.49 0.58 0.63 0.66 1.   0.34 0.5 ]
 [0.44 0.14 0.4  0.23 0.29 0.35 0.36 0.34 1.   0.31]
 [0.47 0.12 0.57 0.38 0.47 0.56 0.55 0.5  0.31 1.  ]]


In [32]:
# create a list of 10 random integers

random_indices = random.sample(range(len(cleaned_documents)), 10)
print("Randomly selected document indices:", random_indices)

Randomly selected document indices: [522, 14, 11, 236, 662, 339, 71, 508, 600, 328]


In [33]:
# Compute similarity scores for 10 random documents

if len(cleaned_documents) >= 10:
    tfidf_random = tfidf_matrix[random_indices]
    similarity_matrix_random = cosine_similarity(tfidf_random)
    print("Similarity matrix for 10 random documents:")
    print(np.round(similarity_matrix_random, 2))
else:
    print("Not enough documents for random similarity analysis.")

Similarity matrix for 10 random documents:
[[1.   0.03 0.1  0.04 0.11 0.03 0.11 0.05 0.11 0.05]
 [0.03 1.   0.07 0.03 0.06 0.02 0.09 0.03 0.07 0.03]
 [0.1  0.07 1.   0.05 0.15 0.05 0.38 0.12 0.16 0.08]
 [0.04 0.03 0.05 1.   0.12 0.02 0.1  0.04 0.12 0.03]
 [0.11 0.06 0.15 0.12 1.   0.07 0.38 0.15 0.84 0.08]
 [0.03 0.02 0.05 0.02 0.07 1.   0.06 0.03 0.07 0.03]
 [0.11 0.09 0.38 0.1  0.38 0.06 1.   0.12 0.34 0.1 ]
 [0.05 0.03 0.12 0.04 0.15 0.03 0.12 1.   0.15 0.04]
 [0.11 0.07 0.16 0.12 0.84 0.07 0.34 0.15 1.   0.09]
 [0.05 0.03 0.08 0.03 0.08 0.03 0.1  0.04 0.09 1.  ]]


### **1.4 Document Creation and Chunking** <font color=red> [5 marks] </font><br>

#### **1.4.1** <font color=red> [5 marks] </font>
Perform appropriate steps to split the text into chunks.

In [34]:
# Process files and generate chunks

# Simple chunking: split each document into chunks of N words (e.g., 300 words)
CHUNK_SIZE = 300
CHUNK_OVERLAP = 50

def chunk_document(text, chunk_size=CHUNK_SIZE, overlap=CHUNK_OVERLAP):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = ' '.join(words[i:i+chunk_size])
        if chunk:
            chunks.append(chunk)
    return chunks

all_chunks = []
chunk_metadata = []
for i, doc in enumerate(cleaned_documents):
    chunks = chunk_document(doc)
    all_chunks.extend(chunks)
    # Store metadata for each chunk
    for chunk in chunks:
        chunk_metadata.append(metadata[i])

print(f"Total chunks created: {len(all_chunks)}")

Total chunks created: 24423


## **2. Vector Database and RAG Chain Creation** <font color=red> [15 marks] </font><br>

### **2.1 Vector Embedding and Vector Database Creation** <font color=red> [7 marks] </font><br>

#### **2.1.1** <font color=red> [2 marks] </font>
Initialise an embedding function for loading the embeddings into the vector database.

Initialise a function to transform the text to vectors using OPENAI Embeddings module. You can also use this function to transform during vector DB creation itself.

In [35]:
# Fetch your OPENAI API Key as an environment variable
import os
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
if not OPENAI_API_KEY:
    raise ValueError('OPENAI_API_KEY environment variable not set.')



In [36]:
# Initialise an embedding function
from langchain_openai import OpenAIEmbeddings

embedding_function = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

#### **2.1.2** <font color=red> [5 marks] </font>
Load the embeddings to a vector database.

Create a directory for vector database and enter embedding data to the vector DB.

In [ ]:
# Add Chunks to vector DB
%pip install -q langchain-community

from langchain_community.vectorstores import Chroma
import shutil
from tqdm import tqdm

VECTOR_DB_DIR = './vector_db'
# Remove previous DB if exists for a clean run
if os.path.exists(VECTOR_DB_DIR):
    shutil.rmtree(VECTOR_DB_DIR)

def batch(iterable, n=100):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

num_batches = (len(all_chunks) + 99) // 100
for chunk_batch, meta_batch in tqdm(zip(batch(all_chunks, 100), batch(chunk_metadata, 100)), total=num_batches, desc="Creating Vector DB", unit="batch"):
    Chroma.from_texts(
        texts=chunk_batch,
        embedding=embedding_function, 
        metadatas=meta_batch,
        persist_directory=VECTOR_DB_DIR 
    )

# Reload the vector DB after all batches are processed
vector_db = Chroma(persist_directory=VECTOR_DB_DIR, embedding_function=embedding_function)
vector_db.persist()
print(f"Vector DB created at {VECTOR_DB_DIR} with {len(all_chunks)} chunks.")

Vector DB created at ./vector_db with 24423 chunks.


### **2.2 Create RAG Chain** <font color=red> [8 marks] </font><br>

#### **2.2.1** <font color=red> [5 marks] </font>
Create a RAG chain.

In [ ]:
# Create a RAG chain
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

# Initialize the LLM
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0)

# Create the RAG chain using the vector DB as retriever
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_db.as_retriever(),
    return_source_documents=True
)

#### **2.2.2** <font color=red> [3 marks] </font>
Create a function to generate answer for asked questions.

Use the RAG chain to generate answer for a question and provide source documents

In [ ]:
# Create a function for question answering

def answer_question(question):
    result = rag_chain({"query": question})
    answer = result['result']
    sources = result.get('source_documents', [])
    return answer, sources


In [ ]:
# Example question
question = "Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; Does the document indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information?"
answer, sources = answer_question(question)
print("Answer:", answer)
if sources:
    print("Sources:")
    for i, src in enumerate(sources):
        print(f"Source {i+1}: {src.metadata.get('file_name', 'Unknown file')}")
else:
    print("Sources: None found or not available.")


C:\Users\hrida\AppData\Local\Temp\ipykernel_16560\3359610982.py:4: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = rag_chain({"query": question})


Answer: Yes, the Non-Disclosure Agreement between CopAcc and ToP Mentors indicates that the Agreement does not grant the Receiving Party any rights to the Confidential Information. The document specifies that the Agreement is not intended to grant the Receiving Party any rights to the confidential information except as expressly set forth in the Agreement.
Sources:
Source 1: thoughtbot-mutual-nda.txt
Source 2: thoughtbot-mutual-nda.txt
Source 3: Evelozcity_20OESA_20NDA.txt
Source 4: NDA-Template-Media-News-Group-inc.txt


## **3. RAG Evaluation** <font color=red> [10 marks] </font><br>

### **3.1 Evaluation and Inference** <font color=red> [10 marks] </font><br>

#### **3.1.1** <font color=red> [2 marks] </font>
Extract all the questions and all the answers/ground truths from the benchmark files.

Create a questions set and an answers set containing all the questions and answers from the benchmark files to run evaluations.

In [38]:
# Create a question set by taking all the questions from the benchmark data
# Also create a ground truth/answer set


# Create a question set by taking all the questions from the benchmark data
# Also create a ground truth/answer set

benchmark_dir = 'rag_legal/benchmarks'  # Update if needed
benchmark_files = ['contractnli.json', 'cuad.json', 'maud.json', 'privacy_qa.json']
questions = []
ground_truths = []

for file in benchmark_files:
    file_path = os.path.join(benchmark_dir, file)
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        for test in data.get('tests', []):
            questions.append(test['query'])
            # Collect all answers for this question (can be multiple snippets)
            answers = [snippet['answer'] for snippet in test.get('snippets', [])]
            ground_truths.append(answers)
    except Exception as e:
        logging.error(f"Error reading benchmark file {file_path}: {e}")

print(f"Total questions loaded: {len(questions)}")

Total questions loaded: 6889


#### **3.1.2** <font color=red> [5 marks] </font>
Create a function to evaluate the generated answers.

Evaluate the responses on *Rouge*, *Ragas* and *Bleu* scores.

In [39]:
# Function to evaluate the RAG pipeline
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from ragas import evaluate as ragas_evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision
from datasets import Dataset
from tqdm import tqdm
import numpy as np

# Evaluate a single prediction against multiple references (ROUGE, BLEU)
def evaluate_single(prediction, references):
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rouge_scores = [scorer.score(ref, prediction)['rougeL'].fmeasure for ref in references]
    rouge = max(rouge_scores) if rouge_scores else 0
    bleu_scores = [sentence_bleu([ref.split()], prediction.split(), smoothing_function=SmoothingFunction().method1) for ref in references]
    bleu = max(bleu_scores) if bleu_scores else 0
    return rouge, bleu

# Function to evaluate all metrics (ROUGE, BLEU, Ragas)
def evaluate_all_metrics(questions, ground_truths, answer_func):
    # ROUGE and BLEU
    results = []
    answers = []
    for i in tqdm(range(len(questions)), desc="Evaluating", unit="question"):
        pred, _ = answer_func(questions[i])
        answers.append(pred)
        refs = ground_truths[i]
        rouge, bleu = evaluate_single(pred, refs)
        results.append({'rouge': rouge, 'bleu': bleu})
    avg_rouge = np.mean([r['rouge'] for r in results])
    avg_bleu = np.mean([r['bleu'] for r in results])
    # Ragas
    ragas_data = {
        "question": questions,
        "contexts": [[" ".join(refs)] for refs in ground_truths],
        "answer": answers,
        "ground_truth": [refs[0] if refs else "" for refs in ground_truths],
    }
    ragas_dataset = Dataset.from_dict(ragas_data)
    ragas_result = ragas_evaluate(
        ragas_dataset,
        metrics=[faithfulness, answer_relevancy, context_precision]
    )
    return avg_rouge, avg_bleu, ragas_result

#### **3.1.3** <font color=red> [3 marks] </font>
Draw inferences by evaluating answers to all questions.

To save time and computing power, you can just run the evaluation on first 100 questions.

In [41]:
# Evaluate the RAG pipeline
# Evaluate on the first 100 questions to save time/resources
eval_questions = questions[:100]
eval_ground_truths = ground_truths[:100]

avg_rouge, avg_bleu, ragas_result = evaluate_all_metrics(eval_questions, eval_ground_truths, answer_question)

print(f"Average ROUGE score: {avg_rouge:.4f}")
print(f"Average BLEU score: {avg_bleu:.4f}")
print("Ragas evaluation results:")
print(ragas_result)

Evaluating:   0%|          | 1/300 [00:02<14:55,  2.99s/it]ERROR:ragas.executor:Exception raised in Job[7]: IndexError(list index out of range)
ERROR:ragas.executor:Exception raised in Job[13]: IndexError(list index out of range)
ERROR:ragas.executor:Exception raised in Job[4]: IndexError(list index out of range)
Evaluating:  10%|█         | 31/300 [01:48<12:15,  2.73s/it]ERROR:ragas.executor:Exception raised in Job[43]: IndexError(list index out of range)
ERROR:ragas.executor:Exception raised in Job[28]: IndexError(list index out of range)
ERROR:ragas.executor:Exception raised in Job[37]: IndexError(list index out of range)
Evaluating:  14%|█▎        | 41/300 [02:29<13:01,  3.02s/it]ERROR:ragas.executor:Exception raised in Job[49]: IndexError(list index out of range)
ERROR:ragas.executor:Exception raised in Job[52]: IndexError(list index out of range)
ERROR:ragas.executor:Exception raised in Job[34]: IndexError(list index out of range)
Evaluating:  19%|█▊        | 56/300 [03:26<13:07,

Average ROUGE score: 0.2383
Average BLEU score: 0.0321
Ragas evaluation results:
{'faithfulness': 0.3610, 'answer_relevancy': 0.9385, 'context_precision': 0.8384}



### The above exceptions are because of picking random 100 samples from various chunks of data. 

## **4. Conclusion** <font color=red> [5 marks] </font><br>

### **4.1 Conclusions and insights** <font color=red> [5 marks] </font><br>

#### **4.1.1** <font color=red> [5 marks] </font>
Conclude with the results here. Include the insights gained about the data, model pipeline, the RAG process and the results obtained.

### **Conclusion and Insights**

**Data Insights:**
- The legal documents were successfully loaded, cleaned, and chunked, resulting in a structured dataset suitable for retrieval tasks.
- Exploratory analysis revealed a wide range of document lengths and vocabulary, with some common legal terms appearing frequently.

**Model Pipeline:**
- The RAG pipeline was built using OpenAI embeddings and a Chroma vector database, enabling efficient retrieval and generation for legal question answering.
- The chunking strategy and metadata preservation ensured relevant context was available for retrieval.

**Evaluation Results:**
- The evaluation functions provided quantitative metrics (ROUGE, BLEU, and Ragas) to assess the quality of generated answers.
- ROUGE and BLEU scores indicated the model's ability to generate text similar to ground truth answers, while Ragas metrics (faithfulness, answer relevancy, context precision) provided a more nuanced view of answer quality.

**RAG Process Insights:**
- The RAG approach effectively combined retrieval and generation, leveraging both the vector database and the language model.
- The use of multiple evaluation metrics highlighted strengths and areas for improvement, such as answer faithfulness and context usage.
